<h2>Planning a Bank Marketing Campaign with SQL<h2>

In this project we'll explore a bank marketing data set with information on the outcome of the last marketing campaign excecuted by a bank to encourage customers to make term deposits. We'll use SQL to analyze a few quick demographic patterns that may help inform our next campaign. The data set can be found on [Kaggle](https://www.kaggle.com/datasets/janiobachmann/bank-marketing-dataset). 

Before we start the analysis, we'll load the data into a SQL database. We'll start by creating an empty SQL database to populate with the Kaggle CSV data:

In [26]:
import sqlite3
from pathlib import Path

Path("bank.db").touch()

conn = sqlite3.connect("bank.db")
c = conn.cursor()

table = """CREATE TABLE bank (
                age int, job text,
                marital text, education text, 
                default_e text, balance int,
                housing text, loan text,
                contact text, day int,
                month text, duration int,
                campaign int, pdays decimal,
                previous int, poutcome text,
                deposit text
                );"""

c.execute(table)

Now, we'll populate the empty database with the Kaggle data:

In [27]:
import pandas as pd

bank = pd.read_csv("bank.csv")

bank.to_sql("bank", conn, if_exists="append", index=False)

11162

Next we'll fetch a few results to test if the table populated correctly:

In [156]:
test_query = """SELECT * FROM bank"""
c.execute(test_query).fetchmany(2)

[(59,
  'admin.',
  'married',
  'secondary',
  'no',
  2343,
  'yes',
  'no',
  'unknown',
  5,
  'may',
  1042,
  1,
  -1,
  0,
  'unknown',
  'yes'),
 (56,
  'admin.',
  'married',
  'secondary',
  'no',
  45,
  'no',
  'no',
  'unknown',
  5,
  'may',
  1467,
  1,
  -1,
  0,
  'unknown',
  'yes')]

Now we'll load the sql extension:

In [77]:
%reload_ext sql
%sql sqlite:///bank.db

<h3>Finding Target Ages & Occupations<h3>

We can begin our analysis now. We'll start by looking at the most common age bracket for the customers in our dataset:

In [137]:
%%sql

WITH 
age_group AS 
(
SELECT age,
        CASE
            WHEN age<30 AND age>0 THEN "Under 30"
            WHEN age<46 AND age>29 THEN  "30-45"
            WHEN age<60 AND age>45 THEN "45-60"
            WHEN age>59 THEN "Over 60"
        END AS age_bracket
FROM bank
),

count_by_age AS
(
SELECT age_bracket, COUNT(*) AS customer_count
  FROM age_group
GROUP BY age_bracket
)

SELECT *
  FROM count_by_age
ORDER BY customer_count DESC;

 * sqlite:///bank.db
Done.


age_bracket,customer_count
30-45,5978
45-60,2853
Under 30,1551
Over 60,780


The most common age bracket is 30-45, followed by 45-60. Let's look at the most common professions for customers in these 2 age brackets:

In [214]:
%%sql
SELECT job, COUNT(*) AS customer_count
  FROM bank
WHERE 29<age<61
GROUP BY job
ORDER BY customer_count DESC
LIMIT 4;

 * sqlite:///bank.db
Done.


job,customer_count
management,2566
blue-collar,1944
technician,1823
admin.,1334


Let's see the success frequency by job, and compare it to the most common jobs for the 30-45 and 45-60 age brackets:

In [213]:
%%sql
SELECT job, COUNT(*) AS customer_count
  FROM bank
WHERE poutcome = "success"
GROUP BY poutcome, job
ORDER BY customer_count DESC
LIMIT 5;

 * sqlite:///bank.db
Done.


job,customer_count
management,292
technician,157
retired,135
admin.,134
blue-collar,90


We can see that management, blue-collar, admin, and technician all appear among the top 5 professions where the previous campaign successfully converted a customer. This is in line the most common professions for the 30-45 and 45-60 age brackets. For these age brackets, we might want to have messaging to specifically target people in the aforementioned professions. 

Additionally, we see retired as the 3rd most frequent occupation for successes, suggesting we might also want messaging tailored to a 60+ retired demographic. 

<h3>Percentage of Customers with Current Loans or Term Deposits Past Campaigns Successfully Converted<h3>

Now let's look at the % of people with loans for whom campaign succeeded. We'll exclude outcomes that are not defined as either failures or successes to focus on figuring out which customer segment to target in the next marketing campaign. 


In [215]:
%%sql
SELECT loan, poutcome, (100*CAST(COUNT(*) AS FLOAT)/(SELECT COUNT(*)
                       FROM bank
                       WHERE (poutcome = "failure") OR (poutcome = "success"))) AS percent
  FROM bank
WHERE (poutcome = "failure") OR (poutcome = "success")
GROUP BY loan, poutcome;

 * sqlite:///bank.db
Done.


loan,poutcome,percent
no,failure,46.107003044802084
no,success,44.36711613745106
yes,failure,7.3075250108742935
yes,success,2.218355806872553


Above we can see that for customers who did not have a loan, the campaign both failed and succeeded for a similar number percentage of the total customers. For customers who did have a loan, similarily, the campaign also both failed and succeeded for a similar number percentage of the total customers. Loans at first glance therefore don't appear to have much of an impact on the the campaign being successful. 

Let's see if term deposits impacted customers for whom the campaign succeeded.

In [195]:
%%sql
SELECT deposit, poutcome, (100*CAST(COUNT(*) AS FLOAT)/(SELECT COUNT(*)
                       FROM bank
                       WHERE (poutcome = "failure") OR (poutcome = "success"))) AS percent
  FROM bank
WHERE (poutcome = "failure") OR (poutcome = "success")
GROUP BY deposit, poutcome;

 * sqlite:///bank.db
Done.


deposit,poutcome,percent
no,failure,26.53327533710309
no,success,4.045237059591127
yes,failure,26.881252718573293
yes,success,42.54023488473249


Above we can the campaign is more likely to succeed with customers who do have a term deposit, versus people who do not.

<h3>Which Bank Balance Range to Target<h3>

Let's investigate if balance has an impact. 

In [173]:
%%sql
SELECT MAX(balance), MIN(balance)
  FROM bank;

 * sqlite:///bank.db
Done.


MAX(balance),MIN(balance)
81204,-6847


In [209]:
%%sql

WITH bal_cat AS (
SELECT poutcome, balance,
        CASE
            WHEN 75000<balance AND balance<82000 THEN  "82K-75K"
            WHEN 50000<balance AND balance<75000 THEN  "75K-50K"
            WHEN 25000<balance AND balance<50000 THEN  "50K-25K"
            WHEN 10000<balance AND balance<25000 THEN  "25K-10K"
            WHEN 5000<balance AND balance<10000 THEN  "10K-5K"
            WHEN 0<balance AND balance<5000 THEN  "Under 5K"
            WHEN balance=0 THEN "no balance"
            WHEN balance<0 THEN "negative balance"
        END AS balance_bracket
FROM bank
WHERE poutcome = "success" OR poutcome = "failure"
)

SELECT balance_bracket, poutcome, COUNT(*)
  FROM bal_cat
GROUP BY balance_bracket, poutcome;

 * sqlite:///bank.db
Done.


balance_bracket,poutcome,COUNT(*)
10K-5K,failure,73
10K-5K,success,62
25K-10K,failure,20
25K-10K,success,30
50K-25K,failure,1
50K-25K,success,3
75K-50K,success,1
82K-75K,success,1
Under 5K,failure,1000
Under 5K,success,922


Most success is with the under \\$5k balance group, but this group also has the most failures. Let's explore whether loans and term deposits in this group can give us insights into why this group is so divided in terms of success. 

In [211]:
%%sql

SELECT poutcome, loan, COUNT(*)
  FROM bank 
WHERE (0<balance AND balance<5000) AND (poutcome = "success" OR poutcome = "failure")
GROUP BY poutcome, loan;

 * sqlite:///bank.db
Done.


poutcome,loan,COUNT(*)
failure,no,866
failure,yes,134
success,no,877
success,yes,45


Both most of the successes and failures in the under \\$5k balance group did not have loans. 

In [212]:
%%sql

SELECT poutcome, deposit, COUNT(*) AS customer_count
  FROM bank 
WHERE (0<balance AND balance<5000) AND (poutcome = "success" OR poutcome = "failure")
GROUP BY poutcome, deposit;

 * sqlite:///bank.db
Done.


poutcome,deposit,customer_count
failure,no,498
failure,yes,502
success,no,81
success,yes,841


In the under \\$5k income group, customers whom the campaign persuaded with were more likely to have a term deposit than not. 

<h3>Conclusion<h3>

The above analysis seems to suggest that for the next marketing campaign the budget and efforts should be weighted more heavily towards people with a positive bank balance under \\$25k with existing term deposits. 

The campaign should also have different messaging strategies for each of following buckets of people: 
•30-45 and 45-60 year olds in management, technician, and admin roles
•60 and above in retirement.